In [51]:
import pandas as pd
from sklearn import tree
import graphviz
pd.options.display.max_rows = 500

In [5]:
df = pd.read_csv('h03_data.csv', header=0)

In [10]:
# get set of unique sequences
unique_seqs = set(df['sequence'])

# create dict for classes based on unique sequences
classes = {seq: c for c, seq in enumerate(unique_seqs)}

# assign class to new column in df

for row in range(0,len(df)):
    df.loc[row,'class'] = classes[df.loc[row,'sequence']]

In [68]:
#df.to_csv('h03_data.csv', header=True)

In [56]:
X = df.iloc[:,0:9]

In [57]:
y = df['class']

In [58]:
clf = tree.DecisionTreeClassifier()

one_hot_data = pd.get_dummies(X)

clf = clf.fit(one_hot_data,y)

In [67]:
dot_data = tree.export_graphviz(clf, out_file=None,
                               feature_names=list(one_hot_data.columns),
                               class_names=df['sequence'],
                               filled=True, rounded=True,
                               special_characters=True)
graph = graphviz.Source(dot_data)
graph.render('tree')

'tree.pdf'